In [89]:
%load_ext autoreload
%autoreload 2
%aimport algorithm_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
from algorithm_utils import *
import general_utils as gu

In [164]:
testing_length = 200
start_index = 6500
pair_df: pd.DataFrame = gu.load_local_data().iloc[start_index:start_index + testing_length].reset_index(drop=True)
pair_df.low.iloc[4] = 0.0387
pair_df

C:\Users\Amin-PC\AppData\Local\Temp\ipykernel_1444\2744840967.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,time,open,high,low,close,candle_color
0,2022-09-28 20:00:00,0.0407,0.0410,0.0407,0.0408,green
1,2022-09-28 21:00:00,0.0409,0.0410,0.0406,0.0406,red
2,2022-09-28 22:00:00,0.0407,0.0408,0.0404,0.0405,red
3,2022-09-28 23:00:00,0.0405,0.0407,0.0402,0.0403,red
4,2022-09-29 00:00:00,0.0402,0.0405,0.0387,0.0400,red
...,...,...,...,...,...,...
195,2022-10-06 23:00:00,0.0406,0.0408,0.0405,0.0408,green
196,2022-10-07 00:00:00,0.0408,0.0412,0.0407,0.0411,green
197,2022-10-07 01:00:00,0.0411,0.0411,0.0409,0.0409,red
198,2022-10-07 02:00:00,0.0410,0.0411,0.0409,0.0410,red


In [175]:
algo = Algo(pair_df, "XEMUSDT")
algo.init_zigzag()
algo.calc_h_o_zigzag()
print(algo.pbos_indices)
print(algo.h_o_indices)
algo.h_o_indices.append(154)
# h_o_zigzag_df = generate_h_o_zigzag(zigzag_df)
# pbos_df = h_o_zigzag_df.loc[h_o_zigzag_df.is_pbos]
# bos_df = find_confirmed_boss(pbos_df, pair_df)
# lpl_df = find_lpls(bos_df, zigzag_df)
# lplb_df = find_lplbs(bos_df, zigzag_df)

# boxes_list = []
# for lpl in lpl_df.itertuples():
#     start_index = lpl.pair_df_index
#     base_candle = Candle.create(pair_df.iloc[lpl.pair_df_index])
#     box_type = "long" if lpl.pivot_type == "valley" else "short"
#     box_to_add = Box(base_candle, b|ox_type)
#     box_to_add.check_box_entries(pair_df)
#     boxes_list.append(box_to_add)

PBOS # 15 broken by candle shadow at index 114
Candle at index 154 broke the last PBOS # 114 with its close price
[15, 114]
[4, 15, 100]


In [176]:
pt = gu.PlottingTool()
pt.draw_candlesticks(pair_df)
pt.draw_zigzag(algo.zigzag_df)
# pt.draw_zigzag(h_o_zigzag_df, title='Higher order Zigzag', color='black')
pt.draw_points_with_label(algo.pair_df.iloc[algo.pbos_indices].index, algo.pair_df.iloc[algo.pbos_indices].high, "PBOS")
pt.draw_points_with_label(algo.zigzag_df[algo.zigzag_df.pair_df_index.isin(algo.h_o_indices)].pair_df_index,
                          algo.zigzag_df[algo.zigzag_df.pair_df_index.isin(algo.h_o_indices)].pivot_value, "HO ZigZag", draw_line=True)
# pt.draw_points_with_label(bos_df.loc[bos_df.pivot_type == "valley"], "BOS-S", color="red")
# pt.draw_points_with_label(lplb_df.loc[lplb_df.pivot_type == "peak"], "LPLB-S", color="orange")
# pt.draw_points_with_label(lpl_df.loc[lpl_df.pivot_type == "peak"], "LPL-S", color="green")
# for box in boxes_list:
#     print(box.id)
#     pt.draw_box(box, pair_df_end_index=pair_df.last_valid_index())
# pt.draw_highlight()
# pt.draw_points_with_label(bos_df.loc[bos_df.pivot_type == "valley"], "BOS-S", color="blue")
pt.show()